In [ ]:
!pip install pillow

In [ ]:
############# MOVE UP AND DOWN AUTOMATICALLY + MOVE WITH MOUSE

import tkinter as tk

# --- functions ---

def move_a(event):
    canvas.coords(a, event.x-50, event.y-50, event.x+50, event.y+50)

def move_b():
    # inform function to use external/global variable
    # because we use `=` to change its value
    global b_speed
    global b_move_down

    canvas.move(b, 0, b_speed)

    # get current position
    x1, y1, x2, y2 = canvas.coords(b)

    # check if you have to change direction
    #if b_speed > 0:
    if b_move_down:
        # if it reachs bottom
        if y2 > 300:
            # change direction
            #b_move_down = False
            b_move_down = not b_move_down
            b_speed = -b_speed
    else:
        # if it reachs top
        if y1 < 0:
            # change direction
            #b_move_down = True
            b_move_down = not b_move_down
            b_speed = -b_speed

    # move again after 25 ms (0.025s)
    root.after(25, move_b)

# --- main ---

# init
root = tk.Tk()

# create canvas
canvas = tk.Canvas(root, width=500, height=300)
canvas.pack()

# create objects
a = canvas.create_rectangle(0, 0, 100, 100, fill='red')
b = canvas.create_rectangle(0, 0, 100, 100, fill='blue')
# create global variables
b_move_down = True
b_speed = 5

# start moving `a` with mouse
canvas.bind("<Motion>", move_a)

# start moving `b` automatically
move_b()

# start program
root.mainloop()

In [ ]:
############# SI NO RECARGAS EL KERNEL DA ERROR PYIMAGE
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk

class MandelbrotViewer:
    def __init__(self, root):
        self.root = root
        self.root.title("Mandelbrot Viewer")

        # Initialize variables
        self.speed = tk.DoubleVar()
        # self.color = tk.DoubleVar()
        self.zoom = tk.DoubleVar()

        # Set default values
        self.speed.set(1.0)
        # self.color.set(50.0)
        self.zoom.set(1.0)

        self.speed_value = self.speed.get()
        self.zoom_value = self.zoom.get()

        # Create sliders with labels
        speed_label = ttk.Label(root, text="Speed:")
        speed_slider = ttk.Scale(root, from_=0.1, to=2.0, variable=self.speed, orient=tk.HORIZONTAL, length=200)

        # color_label = ttk.Label(root, text="Color:")
        # color_slider = ttk.Scale(root, from_=1, to=100, variable=self.color, orient=tk.HORIZONTAL, length=200)

        zoom_label = ttk.Label(root, text="Zoom:")
        zoom_slider = ttk.Scale(root, from_=1, to=5, variable=self.zoom, orient=tk.HORIZONTAL, length=200)

        # Create canvas for displaying the Mandelbrot set
        self.canvas = tk.Canvas(root, width=800, height=600)
        self.canvas.pack()

        # Keep a reference to the PhotoImage in the canvas object
        self.tk_image = None

        # Bind slider updates to the function that redraws the Mandelbrot set
        speed_slider.bind("<Motion>", self.update_mandelbrot)
        # color_slider.bind("<B1-Motion>", self.update_mandelbrot)
        zoom_slider.bind("<Motion>", self.update_mandelbrot)

        # Bind to <ButtonRelease-1> event to trigger update when mouse button is released
        speed_slider.bind("<ButtonRelease-1>", self.update_mandelbrot)
        # color_slider.bind("<ButtonRelease-1>", self.update_mandelbrot)
        zoom_slider.bind("<ButtonRelease-1>", self.update_mandelbrot)

        # Pack sliders with labels
        speed_label.pack()
        speed_slider.pack(pady=10)

        # color_label.pack()
        # color_slider.pack(pady=10)

        zoom_label.pack()
        zoom_slider.pack(pady=10)

        # Initial Mandelbrot parameters
        self.cx, self.cy = -0.5, 0.0

        # Start the animation
        self.animate()

    def mandelbrot(self, x, y, max_iter):
        """
        Mandelbrot set computation.
        """
        zx, zy = x, y
        c = complex(self.cx, self.cy)

        for i in range(max_iter):
            if abs(zx + zy) > 16.0:
                return i
            zx, zy = zx * zx - zy * zy + x, 2.0 * zx * zy + y

        return max_iter

    def update_mandelbrot(self, event):
        """
        Update Mandelbrot set based on slider values.
        """
        self.speed_value = self.speed.get()
        # self.color = int(self.color.get())
        self.zoom_value = self.zoom.get()


    def animate(self):
        """
        Animate the Mandelbrot set based on slider values.
        """
        # max_iter = 100 + int(self.color.get())
        max_iter = 100
        width, height = 800, 600

        base_color = (71, 173, 199)

        image = Image.new("RGB", (width, height), "white")
        pixels = image.load()

        for x in range(width):
            for y in range(height):
                mandel_value = self.mandelbrot((x - width / 2) / (0.5 * self.zoom_value* width),
                                               (y - height / 2) / (0.5 * self.zoom_value * height),
                                               max_iter)
                # color = (mandel_value % 8 * 32, mandel_value % 16 * 16, mandel_value % 32 * 8)
                # Set color intensity based on mandelbrot value
                intensity = int((mandel_value / max_iter) * 255)
                color = (
                    min(int(base_color[0] * intensity / 255), 255),
                    min(int(base_color[1] * intensity / 255), 255),
                    min(int(base_color[2] * intensity / 255), 255)
                )
                pixels[x, y] = color

        # Create a new PhotoImage if it doesn't exist or update the existing one
        if self.tk_image is None:
            self.tk_image = ImageTk.PhotoImage(image)
            self.canvas.create_image(0, 0, anchor=tk.NW, image=self.tk_image, tags="background")
        else:
            self.tk_image.paste(image)
            self.canvas.update()

        # Adjust the Mandelbrot parameters based on the sliders
        self.cx += 0.005 * float(self.speed_value)
        self.cy += 0.005 * float(self.speed_value)

        # Schedule the next animation frame
        self.root.after(50, self.animate)

if __name__ == "__main__":
    root = tk.Tk()
    app = MandelbrotViewer(root)
    root.mainloop()


In [4]:
############# MOVE AROUND + MOVE WITH MOUSE
import tkinter as tk

# --- functions ---

def move_a(event):
    canvas.coords(a, event.x-50, event.y-50, event.x+50, event.y+50)

def move_b():
    # inform function to use external/global variable
    # because we use `=` to change its value
    global b_speed_x
    global b_speed_y
    global b_direction

    canvas.move(b, b_speed_x, b_speed_y)

    # get current position
    x1, y1, x2, y2 = canvas.coords(b)

    if b_direction == 'down':
        if y2 >= 300:
            b_direction = 'right'
            b_speed_x = 5
            b_speed_y = 0
    elif b_direction == 'up':
        if y1 <= 0:
            b_direction = 'left'
            b_speed_x = -5
            b_speed_y = 0
    elif b_direction == 'right':
        if x2 >= 500:
            b_direction = 'up'
            b_speed_x = 0
            b_speed_y = -5
    elif b_direction == 'left':
        if x1 <= 0:
            b_direction = 'down'
            b_speed_x = 0
            b_speed_y = 5

    # move again after 25 ms (0.025s)
    root.after(25, move_b)

# --- main ---

# init
root = tk.Tk()

# create canvas
canvas = tk.Canvas(root, width=500, height=300)
canvas.pack()

# create objects
a = canvas.create_rectangle(0, 0, 100, 100, fill='red')
b = canvas.create_rectangle(0, 0, 100, 100, fill='blue')
# create global variables
b_direction = 'down'
b_speed_x = 0
b_speed_y = 5

# start moving `a` with mouse
canvas.bind("<Motion>", move_a)

# start moving `b` automatically
move_b()

# start program
root.mainloop()

In [5]:
import tkinter as tk

# --- functions ---

def move_a(event):
    canvas.coords(a, event.x-25, event.y-25, event.x+25, event.y+25)

def move_b():
    # inform function to use external/global variable
    # because we use `=` to change its value
    global b_speed_x
    global b_speed_y
    global b_direction

    canvas.move(b, b_speed_x, b_speed_y)

    # get current position
    x1, y1, x2, y2 = canvas.coords(b)

    if b_direction == 'down':
        if y2 >= 300:
            b_direction = 'right'
            b_speed_x = 5
            b_speed_y = 0
    elif b_direction == 'up':
        if y1 <= 0:
            b_direction = 'left'
            b_speed_x = -5
            b_speed_y = 0
    elif b_direction == 'right':
        if x2 >= 500:
            b_direction = 'up'
            b_speed_x = 0
            b_speed_y = -5
    elif b_direction == 'left':
        if x1 <= 0:
            b_direction = 'down'
            b_speed_x = 0
            b_speed_y = 5

    # move again after 25 ms (0.025s)
    root.after(25, move_b)

# --- main ---

# init
root = tk.Tk()

# create canvas
canvas = tk.Canvas(root, width=500, height=300)
canvas.pack()

# create objects
a = canvas.create_oval(0, 0, 15, 15, fill='red')  # Circle instead of rectangle for `a`
b = canvas.create_rectangle(0, 0, 100, 100, fill='blue')

# create global variables
b_direction = 'down'
b_speed_x = 0
b_speed_y = 5

# start moving `a` with mouse
canvas.bind("<Motion>", move_a)

# start moving `b` automatically
move_b()

# start program
root.mainloop()
